In [1]:
import sys
import time
import heapq
import math
import gc
import numpy as np
import pandas as pd
from collections import defaultdict
import surprise as sp
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer
from sklearn.model_selection import KFold
from torchvision import datasets, transforms
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
print (torch.cuda.is_available())
print (torch.version.cuda)
print (torch.cuda.get_device_name(torch.cuda.current_device()))

True
9.0.176
GeForce RTX 2080 Ti


In [2]:
#Evaluation of metrics
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0
#dataset
class DataSet_1M(object):
    def __init__(self, negNum=1):
        self.trainList, self.shape = self._getTrainData()
        self.trainDict = self._getTrainDict()
        self.trainMat = self._getTrainMatrix()
        self.trainset = self._getInstances(negNum)#sample negative samples
        self.testset = self._getTest()
        
    def _getTrainData(self):
        data = []
        filePath = '/data/fjsdata/ResData2019/BayesianMF/ml-1m.train.rating'
        u = 0
        i = 0
        maxr = 0.0
        with open(filePath, 'r') as f:
            for line in f:
                if line:
                    lines = line[:-1].split("\t")
                    user = int(lines[0])
                    movie = int(lines[1])
                    score = float(lines[2])
                    data.append((user, movie, score))
                    if user > u:u = user
                    if movie > i:i = movie
                    if score > maxr:maxr = score
        self.maxRate = maxr
        print("Loading Success!\n"
                  "Data Info:\n"
                  "\tUser Num: {}\n"
                  "\tItem Num: {}\n"
                  "\tData Size: {}\n"
                  "\tSparsity: {}".format(u+1, i+1, len(data), len(data)/((u+1)*(i+1))))
        return data, [u+1, i+1]

    def _getTrainDict(self):
        dataDict = {}
        for i in self.trainList:
            dataDict[(i[0], i[1])] = i[2]
        return dataDict

    def _getTrainMatrix(self):
        train_matrix = np.zeros([self.shape[0], self.shape[1]], dtype=np.float32)
        for i in self.trainList:
            user = i[0]
            movie = i[1]
            rating = i[2]
            train_matrix[user][movie] = rating
        return np.array(train_matrix)

    def _getInstances(self, negNum):
        trainset = []
        for i in self.trainList:
            #trainset.append([i[0],i[1],i[2]])
            trainset.append([i[0],i[1],1.0])
            for t in range(negNum):
                j = np.random.randint(self.shape[1])
                while (i[0], j) in self.trainDict:
                    j = np.random.randint(self.shape[1])
                trainset.append([i[0],j,0.0])
        print ('The length of Trainset: %d'%(len(trainset)))
        return trainset

    def _getTest(self):
        #loading data
        testset = []
        filePath = '/data/fjsdata/ResData2019/BayesianMF/ml-1m.test.negative'
        with open(filePath, 'r') as fd:
            line = fd.readline()
            while line != None and line != '':
                arr = line.split('\t')
                u = eval(arr[0])[0]
                testset.append([u, eval(arr[0])[1], 1.0])#first is one postive item
                for i in arr[1:]:
                    testset.append([u, int(i), 0.0]) #99 negative items
                line = fd.readline()
        print ('The length of Testset: %d'%(len(testset)))
        return testset

In [12]:
ds1m = DataSet_1M(negNum=1)
trainset = ds1m.trainset
testset = ds1m.testset
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(pd.DataFrame(trainset),reader)
trainset_svd = spdata.build_full_trainset()
for k in [5,10,15,20]:#iterations epoches
    algo = sp.SVD(n_factors=k, n_epochs=20, lr_all=0.001, reg_all=0.01 )#
    tstart = time.time()
    algo.fit(trainset_svd)
    elapsed = time.time() - tstart    
    print('Completed training the SVD model in %d seconds' % int(elapsed))
    predictions = algo.test(testset)#testset include one positive and 99 negtive sample of every user.
    user_iid_true_est = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_iid_true_est[uid].append((iid, true_r, est))
    hits = []
    ndcgs = []
    for uid, iid_ratings in user_iid_true_est.items():
        # Sort user ratings by estimated value
        #iid_ratings.sort(key=lambda x: x[2], reverse=True) #sorted by est
        scorelist = []
        positem = -1
        for iid, ture_r, est in iid_ratings:
            if positem == -1: positem=iid #one positive item in first
            scorelist.append([iid,est])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, positem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, positem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("HR@%d=%.6f, NDCG@%d=%.6f" % (k, hitratio, k, ndcg))

Loading Success!
Data Info:
	User Num: 6040
	Item Num: 3706
	Data Size: 994169
	Sparsity: 0.04441379291858915
The length of Trainset: 1988338
The length of Testset: 604000
Completed training the SVD model in 42 seconds
HR@5=0.444040, NDCG@5=0.246101
Completed training the SVD model in 49 seconds
HR@10=0.443709, NDCG@10=0.245849
Completed training the SVD model in 56 seconds
HR@15=0.446192, NDCG@15=0.246611
Completed training the SVD model in 64 seconds
HR@20=0.441225, NDCG@20=0.242097


In [23]:
class NeuralMatrixFactorization(nn.Module):
    def __init__(self, input_dim_u, input_dim_i, factors_dim_k, num_units=[512]):
        super(NeuralMatrixFactorization, self).__init__()
        
        self.input_dim_u = input_dim_u #user vector
        self.input_dim_i = input_dim_i #item vector
        self.factors_dim_k = factors_dim_k #latent factors vector
        
        # network with three hidden and k output layer
        self.layer1_u = nn.Linear(input_dim_u, num_units[0])
        self.layer2_u = nn.Linear(num_units[0], factors_dim_k)
        
        self.layer1_i = nn.Linear(input_dim_i, num_units[0])
        self.layer2_i = nn.Linear(num_units[0], factors_dim_k)
        
        # activation to be used between hidden layers
        #self.activation = nn.ReLU(inplace = True)
    
    def forward(self, x_u, x_i):
    
        x_u = x_u.view(-1, self.input_dim_u)
        x_i = x_i.view(-1, self.input_dim_i)
        #layer1
        x_u = self.layer1_u(x_u)
        x_i = self.layer1_i(x_i)
        #layer2
        x_u = self.layer2_u(x_u)
        x_i = self.layer2_i(x_i)
        
        output = torch.sum(torch.mul(x_u,x_i),1)#pxq     
        #output = torch.sigmoid(output)
        return output
    
class NMF_Model_Wrapper:
    def __init__(self, network, learn_rate=1e-2):
        
        self.learn_rate = learn_rate
        self.network = network
        self.network.cuda()
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr = self.learn_rate)
        self.loss_func = nn.BCELoss()#nn.BCEWithLogitsLoss()#nn.MSELoss()
    
    def fit(self, x_u, x_i, y):
        x_u = torch.from_numpy(np.array(x_u)).type(torch.FloatTensor).cuda()
        x_i = torch.from_numpy(np.array(x_i)).type(torch.FloatTensor).cuda()
        y = torch.from_numpy(np.array(y)).type(torch.FloatTensor).cuda()
        # reset gradient and total loss
        self.optimizer.zero_grad()
        output = self.network(x_u, x_i)
        output = torch.sigmoid(output)
        fit_loss = self.loss_func(output, y)
        
        fit_loss.backward()
        self.optimizer.step()

        return fit_loss

#dataset
ds_1m = DataSet_1M(negNum=1)
trainset = ds_1m.trainset
trainMat = ds_1m.trainMat
testset = ds_1m.testset
shape = ds_1m.shape
#training
sample_size = 1
batchSize = 2000
for k in [5,10,15,20]:  
    #NMF training
    torch.cuda.empty_cache()
    torch.cuda.synchronize()     
    net = NMF_Model_Wrapper(network=NeuralMatrixFactorization(input_dim_u=shape[1], input_dim_i = shape[0],factors_dim_k=k))
    best_net, best_loss = None, float('inf')
    for epoch in range(sample_size):
        shuffled_idx = np.random.permutation(np.arange(len(trainset)))
        trainset = np.array(trainset)[shuffled_idx].tolist()
        num_batches = len(trainset) // batchSize + 1 
        total_loss = []
        for i in range(num_batches):#batch
            min_idx = i * batchSize
            max_idx = np.min([len(trainset), (i+1)*batchSize])
            train_batch = trainset[min_idx: max_idx]
            x_u, x_i, y = [], [], []
            for uu,ii,rr in train_batch:
                x_u.append(trainMat[int(uu),:])
                x_i.append(trainMat[:,int(ii)])
                y.append(float(rr))
            _loss = net.fit(np.array(x_u), np.array(x_i), np.array(y))
            sys.stdout.write('\r {} / {} : loss = {}'.format(i, num_batches, float('%0.6f'%_loss.item())))
            sys.stdout.flush()
            total_loss.append(_loss.item())
        print("Eopch: %5d total_loss = %.6f" % (epoch + 1, np.mean(total_loss)))
        if np.mean(total_loss) < best_loss:
            best_loss = np.mean(total_loss)
            best_net = copy.deepcopy(net.network)
    #torch.save(best_net, "/data/tmpexec/BDMF_torch")
    #best_net = torch.load("/data/tmpexec/BDMF_torch").eval()
    #best_net = torch.load("/data/tmpexec/BDMF_torch").to('cuda:0')
    hits = []
    ndcgs = []
    for c in range(0,shape[0]):#6040
        scorelist = []
        gtItem = -1
        x_u, x_i, y_i = [], [], []
        for uu,ii,rr in testset[c*100:(c+1)*100]:#604000
            if rr == 1.0: 
                gtItem = ii
            x_u.append(np.array(trainMat[int(uu),:]))
            x_i.append(np.array(trainMat[:,int(ii)]))
            y_i.append(ii)
        x_u = torch.from_numpy(np.array(x_u)).type(torch.FloatTensor).cuda()
        x_i = torch.from_numpy(np.array(x_i)).type(torch.FloatTensor).cuda()
        output = best_net(x_u, x_i)
        output = output.cpu().data.numpy().tolist()
        for j in range(len(y_i)):
            scorelist.append([y_i[j],output[j]])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#topn=10
        hr = getHitRatio(ranklist, gtItem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, gtItem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("HR@%d=%.6f, NDCG@%d=%.6f" % (k, hitratio, k, ndcg))

Loading Success!
Data Info:
	User Num: 6040
	Item Num: 3706
	Data Size: 994169
	Sparsity: 0.04441379291858915
The length of Trainset: 1988338
The length of Testset: 604000
 429 / 995 : loss = 0.693147

KeyboardInterrupt: 

In [ ]:
def log_gaussian_loss(output, target, sigma, no_dim, sum_reduce=True):
    exponent = -0.5*(target - output)**2/sigma**2
    log_coeff = -no_dim*torch.log(sigma) - 0.5*no_dim*np.log(2*np.pi)
    
    if sum_reduce:
        return -(log_coeff + exponent).sum()
    else:
        return -(log_coeff + exponent)
    
class gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def loglik(self, weights):
        exponent = -0.5*(weights - self.mu)**2/self.sigma**2
        log_coeff = -0.5*(np.log(2*np.pi) + 2*np.log(self.sigma))
        
        return (exponent + log_coeff).sum()
    
class BayesLinear_Normalq(nn.Module):
    def __init__(self, input_dim, output_dim, prior):
        super(BayesLinear_Normalq, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.prior = prior
        
        self.weight_mus = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-0.01, 0.01))
        self.weight_rhos = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-3, -3))
        
    def forward(self, x):
        # sample gaussian noise for each weight
        weight_epsilons = Variable(self.weight_mus.data.new(self.weight_mus.size()).normal_())      
        # calculate the weight stds from the rho parameters
        weight_stds = torch.log(1 + torch.exp(self.weight_rhos))
        # calculate samples from the posterior from the sampled noise and mus/stds
        weight_sample = self.weight_mus + weight_epsilons*weight_stds
            
        torch.cuda.synchronize()
        output = torch.mm(x, weight_sample)
            
        # computing the KL loss term
        #reference: https://github.com/jojonki/AutoEncoders/blob/master/kl_divergence_between_two_gaussians.pdf
        prior_cov, varpost_cov = self.prior.sigma**2, weight_stds**2
        KL_loss = 0.5*(torch.log(prior_cov/varpost_cov)).sum() - 0.5*weight_stds.numel()
        KL_loss = KL_loss + 0.5*(varpost_cov/prior_cov).sum()
        KL_loss = KL_loss + 0.5*((self.weight_mus - self.prior.mu)**2/prior_cov).sum()
            
        return output, KL_loss
    
class BBP_Model(nn.Module):
    def __init__(self, input_dim_u, input_dim_i, factors_dim_k, num_units=[512]):
        super(BBP_Model, self).__init__()
        
        self.input_dim_u = input_dim_u #user vector
        self.input_dim_i = input_dim_i #item vector
        self.factors_dim_k = factors_dim_k #latent factors vector
        
        # network with three hidden and k output layer
        self.layer1_u = BayesLinear_Normalq(input_dim_u, num_units[0], gaussian(0, 3))
        self.layer2_u = BayesLinear_Normalq(num_units[0], factors_dim_k, gaussian(0, 3))
        
        self.layer1_i = BayesLinear_Normalq(input_dim_i, num_units[0], gaussian(0, 3))
        self.layer2_i = BayesLinear_Normalq(num_units[0], factors_dim_k, gaussian(0, 3))
        
        # activation to be used between hidden layers
        self.activation = nn.ReLU(inplace = True)
        # noise
        self.log_noise = nn.Parameter(torch.cuda.FloatTensor([3]))
    
    def forward(self, x_u, x_i):
        
        KL_loss_total = 0
        x_u = x_u.view(-1, self.input_dim_u)
        x_i = x_i.view(-1, self.input_dim_i)
        #layer1
        x_u, KL_loss_u = self.layer1_u(x_u)
        x_u = self.activation(x_u)
        x_i, KL_loss_i = self.layer1_i(x_i)
        x_i = self.activation(x_i)
        KL_loss_total = KL_loss_total + KL_loss_u + KL_loss_i
        #layer2
        x_u, KL_loss_u = self.layer2_u(x_u)
        x_u = self.activation(x_u)
        x_i, KL_loss_i = self.layer2_i(x_i)
        x_i = self.activation(x_i)
        KL_loss_total = KL_loss_total + KL_loss_u + KL_loss_i
        #pxq
        output = torch.sum(torch.mul(x_u,x_i),1)
        
        return output, KL_loss_total
    
class BBP_Model_Wrapper:
    def __init__(self, network, learn_rate=1e-2):
        
        self.learn_rate = learn_rate
        self.network = network
        self.network.cuda()
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr = self.learn_rate)
        self.loss_func = log_gaussian_loss#nn.MSELoss() 
    
    def fit(self, x_u, x_i, y, no_samples):
        len_sql = y.shape[0]
        x_u = torch.from_numpy(np.array(x_u)).type(torch.FloatTensor).cuda()
        x_i = torch.from_numpy(np.array(x_i)).type(torch.FloatTensor).cuda()
        y = torch.from_numpy(np.array(y)).type(torch.FloatTensor).cuda()
        # reset gradient and total loss
        self.optimizer.zero_grad()
        fit_loss_total = 0
        KL_loss_total = 0
        for i in range(no_samples):
            output, KL_loss = self.network(x_u, x_i)
            KL_loss_total = KL_loss_total + KL_loss
            # calculate fit loss based on mean and standard deviation of output
            fit_loss = self.loss_func(output, y, self.network.log_noise.exp(), 1) 
            fit_loss_total = fit_loss_total + fit_loss
        
        total_loss = (fit_loss_total + KL_loss_total)/(no_samples*len_sql)
        total_loss.backward()
        self.optimizer.step()

        return total_loss
    
#training model
num_epochs = 10
batchSize = 10000
#ds_1m = DataSet_1M()
#trainset = ds_1m.trainset
#trainMat = ds_1m.trainMat
#testset = ds_1m.testset
#shape = ds_1m.shape
for k in [5,10,15,20]:  
    torch.cuda.empty_cache()
    net = BBP_Model_Wrapper(network=BBP_Model(input_dim_u=shape[1], input_dim_i = shape[0],factors_dim_k=k))
    best_net, best_loss = None, float('inf')
    for epoch in range(num_epochs): #iteration
        shuffled_idx = np.random.permutation(np.arange(len(trainset)))
        trainset = np.array(trainset)[shuffled_idx].tolist()
        num_batches = len(trainset) // batchSize + 1 
        total_loss = []
        for i in range(num_batches):#batch
            min_idx = i * batchSize
            max_idx = np.min([len(trainset), (i+1)*batchSize])
            train_batch = trainset[min_idx: max_idx]
            x_u, x_i, y = [], [], []
            for uu,ii,rr in train_batch:
                x_u.append(trainMat[int(uu),:])
                x_i.append(trainMat[:,int(ii)])
                y.append(float(rr))
            _loss = net.fit(np.array(x_u), np.array(x_i), np.array(y), no_samples=10)
            sys.stdout.write('\r {} / {} : loss = {}'.format(i, num_batches, float('%0.6f'%_loss.item())))
            sys.stdout.flush()
            total_loss.append(_loss.item())
        print("Epoch: %5d total_loss = %.6f" % (epoch + 1, np.mean(total_loss)))
        if np.mean(total_loss) < best_loss:
            best_loss = np.mean(total_loss)
            best_net = copy.deepcopy(net.network)
    #torch.save(best_net, "/data/tmpexec/BDMF_torch")
    #best_net = torch.load("/data/tmpexec/BDMF_torch").eval()
    #best_net = torch.load("/data/tmpexec/BDMF_torch").to('cuda:0')
    #performance   
    hits = []
    ndcgs = []
    for c in range(0,shape[0]):#6040
        scorelist = []
        gtItem = -1
        x_u, x_i, y_i = [], [], []
        for uu,ii,rr in testset[c*100:(c+1)*100]:#604000
            if rr == 1.0: 
                gtItem = ii #real hit item
            x_u.append(np.array(trainMat[int(uu),:]))
            x_i.append(np.array(trainMat[:,int(ii)]))
            y_i.append(ii)
        x_u = torch.from_numpy(np.array(x_u)).type(torch.FloatTensor).cuda()
        x_i = torch.from_numpy(np.array(x_i)).type(torch.FloatTensor).cuda()
        output,KL_loss = best_net(x_u, x_i)
        output = output.cpu().data.numpy().tolist()
        for j in range(len(y_i)):
            scorelist.append([y_i[j],output[j]])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)##topn=10
        hr = getHitRatio(ranklist, gtItem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, gtItem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("HR@%d=%.6f, NDCG@%d=%.6f" % (k, hitratio, k, ndcg))

In [3]:
class BayesianMatrixFactorization():
    """
    Bayesian Matrix Factorization model
    R = PxQ
    p ~ N(p|0, alpha^(-1)I)
    q ~ N(q|0, alpha^(-1)I)
    r = p @ q
    t ~ N(r|p @ q, beta^(-1))
    """

    def __init__(self, alpha_p:float=1., alpha_q:float=1., beta:float=1.):
        """
        ----------
        n_u, n_i: the number of users and items, respectively.
        k : the number of latent factors
        """
        self.alpha_p = alpha_p
        self.alpha_q = alpha_q
        self.beta = beta
        #posterior of p,q 
        self.pos_mean_p = None
        self.pos_precision_p = None
        self.pos_mean_q = None
        self.pos_precision_q = None

    def fit(self, R:np.ndarray, k:int=5):
        """
        bayesian update of parameters given training dataset
        Parameters
        ----------
        R : (u,i) np.ndarray
            training data independent variable, u is the number of users, i is the number of items.
        k : int, the number of latent factors.
        """
        #1. generate matrices P, Q
        P = np.random.normal(0,self.alpha_p,(R.shape[0],k))#uxk
        Q = np.random.normal(0,self.alpha_q,(R.shape[1],k))#ixk
        #2.calculate the posterior with analytical solution
        self.pos_precision_p = self.alpha_p + self.beta * Q @ Q.T # ixi
        self.pos_mean_p = self.beta * R @ np.linalg.inv(self.pos_precision_p) @ Q # uxi,ixi,ixk -> uxk
        self.pos_precision_q = self.alpha_q + self.beta * P @ P.T # uxu
        self.pos_mean_q = self.beta * R.T @ np.linalg.inv(self.pos_precision_q) @ P # ixu,uxu,uxk -> ixk

    def predict(self, sample_size:int=None):
        """
        return mean  of predictive distribution
        Parameters
        ----------
        sample_size : int, optional
            number of samples to draw from the predictive distribution
            (the default is None, no sampling from the distribution)
        Returns
        -------
        R_pred : (u,i) np.ndarray
            mean of the predictive distribution
        R_pred_sample : (u,i,sample_size) np.ndarray
            samples from the predictive distribution
        """
        if sample_size is not None:
            R_sample = []
            for i in range(sample_size):
                p_sample, q_sample = [], []
                for k in range(self.pos_mean_p.shape[1]):#latent factors    
                    mean_p = self.pos_mean_p[:,k]
                    mean_q = self.pos_mean_q[:,k]
                    p_sample_k = np.random.multivariate_normal(mean_p, np.linalg.inv(self.pos_precision_q), size=1)
                    q_sample_k = np.random.multivariate_normal(mean_q, np.linalg.inv(self.pos_precision_p), size=1)
                    p_sample.append(p_sample_k.flatten())
                    q_sample.append(q_sample_k.flatten())
                R_sample.append(np.dot(np.array(p_sample).T, np.array(q_sample)))
            return  R_sample #uxi
        
        R_pred = self.pos_mean_p @ self.pos_mean_q.T #R = PxQ
        return R_pred #uxi
    
class NeuralMatrixFactorization(nn.Module):
    def __init__(self, input_dim_u, input_dim_i, factors_dim_k, num_units=[512]):
        super(NeuralMatrixFactorization, self).__init__()
        
        self.input_dim_u = input_dim_u #user vector
        self.input_dim_i = input_dim_i #item vector
        self.factors_dim_k = factors_dim_k #latent factors vector
        
        # network with three hidden and k output layer
        self.layer1_u = nn.Linear(input_dim_u, num_units[0])
        self.layer2_u = nn.Linear(num_units[0], factors_dim_k)
        
        self.layer1_i = nn.Linear(input_dim_i, num_units[0])
        self.layer2_i = nn.Linear(num_units[0], factors_dim_k)
        
        # activation to be used between hidden layers
        self.activation = nn.ReLU(inplace = True)
    
    def forward(self, x_u, x_i):
    
        x_u = x_u.view(-1, self.input_dim_u)
        x_i = x_i.view(-1, self.input_dim_i)
        #layer1
        x_u = self.layer1_u(x_u)
        x_u = self.activation(x_u)
        x_i = self.layer1_i(x_i)
        x_i = self.activation(x_i)
        #layer2
        x_u = self.layer2_u(x_u)
        x_u = self.activation(x_u)
        x_i = self.layer2_i(x_i)
        x_i = self.activation(x_i)
        
        output = torch.sum(torch.mul(x_u,x_i),1)#pxq
        #output = torch.sigmoid(output)
        return output
    
class NMF_Model_Wrapper:
    def __init__(self, network, learn_rate=1e-2):
        
        self.learn_rate = learn_rate
        self.network = network
        self.network.cuda()
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr = self.learn_rate)
        self.loss_func = nn.BCELoss()#nn.BCEWithLogitsLoss()#nn.MSELoss()
    
    def fit(self, x_u, x_i, y):
        x_u = torch.from_numpy(np.array(x_u)).type(torch.FloatTensor).cuda()
        x_i = torch.from_numpy(np.array(x_i)).type(torch.FloatTensor).cuda()
        y = torch.from_numpy(np.array(y)).type(torch.FloatTensor).cuda()
        # reset gradient and total loss
        self.optimizer.zero_grad()
        output = self.network(x_u, x_i)
        output = torch.sigmoid(output)
        fit_loss = self.loss_func(output, y)
        
        fit_loss.backward()
        self.optimizer.step()

        return fit_loss

In [ ]:
#dataset
ds_1m = DataSet_1M(negNum=0)
trainset = ds_1m.trainset
trainMat = ds_1m.trainMat
testset = ds_1m.testset
shape = ds_1m.shape
#training
sample_size = 2 # samples 
batchSize = 2000
for k in [5,10,15,20]:  
    #get approximate matrice
    bmf = BayesianMatrixFactorization()
    bmf.fit(R=trainMat, k=k)
    R_sample = bmf.predict(sample_size=sample_size)
    #NMF training
    torch.cuda.empty_cache()
    torch.cuda.synchronize()     
    net = NMF_Model_Wrapper(network=NeuralMatrixFactorization(input_dim_u=shape[1], input_dim_i = shape[0],factors_dim_k=k))
    best_net, best_loss = None, float('inf')
    for iSpl, iR in enumerate(R_sample):
        shuffled_idx = np.random.permutation(np.arange(len(trainset)))
        trainset = np.array(trainset)[shuffled_idx].tolist()
        num_batches = len(trainset) // batchSize + 1 
        total_loss = []
        for i in range(num_batches):#batch
            min_idx = i * batchSize
            max_idx = np.min([len(trainset), (i+1)*batchSize])
            train_batch = trainset[min_idx: max_idx]
            x_u, x_i, y = [], [], []
            for uu,ii,rr in train_batch:
                x_u.append(iR[int(uu),:])
                x_i.append(iR[:,int(ii)])
                y.append(float(rr))
            _loss = net.fit(np.array(x_u), np.array(x_i), np.array(y))
            sys.stdout.write('\r {} / {} : loss = {}'.format(i, num_batches, float('%0.6f'%_loss.item())))
            sys.stdout.flush()
            total_loss.append(_loss.item())
        print("Sample: %5d total_loss = %.6f" % (iSpl + 1, np.mean(total_loss)))
        if np.mean(total_loss) < best_loss:
            best_loss = np.mean(total_loss)
            best_net = copy.deepcopy(net.network)
    #torch.save(best_net, "/data/tmpexec/BDMF_torch")
    #best_net = torch.load("/data/tmpexec/BDMF_torch").eval()
    #best_net = torch.load("/data/tmpexec/BDMF_torch").to('cuda:0')
    hits = []
    ndcgs = []
    for c in range(0,shape[0]):#6040
        scorelist = []
        gtItem = -1
        x_u, x_i, y_i = [], [], []
        for uu,ii,rr in testset[c*100:(c+1)*100]:#604000
            if rr == 1.0: 
                gtItem = ii
            x_u.append(np.array(trainMat[int(uu),:]))
            x_i.append(np.array(trainMat[:,int(ii)]))
            y_i.append(ii)
        x_u = torch.from_numpy(np.array(x_u)).type(torch.FloatTensor).cuda()
        x_i = torch.from_numpy(np.array(x_i)).type(torch.FloatTensor).cuda()
        output = best_net(x_u, x_i)
        output = output.cpu().data.numpy().tolist()
        for j in range(len(y_i)):
            scorelist.append([y_i[j],output[j]])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#topn=10
        hr = getHitRatio(ranklist, gtItem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, gtItem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("HR@%d=%.6f, NDCG@%d=%.6f" % (k, hitratio, k, ndcg))

Loading Success!
Data Info:
	User Num: 6040
	Item Num: 3706
	Data Size: 994169
	Sparsity: 0.04441379291858915
The length of Trainset: 994169
The length of Testset: 604000


/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: covariance is not positive-semidefinite.
/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: covariance is not positive-semidefinite.


 497 / 498 : loss = 0.184566Sample:     1 total_loss = 0.189159
 497 / 498 : loss = 0.176363Sample:     2 total_loss = 0.193778
HR@5=0.327980, NDCG@5=0.176795
 497 / 498 : loss = 0.045116Sample:     1 total_loss = 0.054246
 497 / 498 : loss = 0.094334Sample:     2 total_loss = 0.083603
HR@10=0.266225, NDCG@10=0.140981
 497 / 498 : loss = 0.008203Sample:     1 total_loss = 0.002754
 497 / 498 : loss = 0.000693Sample:     2 total_loss = 0.001895
HR@15=0.297351, NDCG@15=0.150598
 497 / 498 : loss = 0.000693Sample:     1 total_loss = 0.001021
 497 / 498 : loss = 0.000347Sample:     2 total_loss = 0.000666
